In [1]:
import pandas as pd
# from tabulate import tabulate
import numpy as np
# import bnlearn as bn
import pgmpy
import itertools
from scipy.stats import chi2_contingency #check for independence between 2 variables
from scipy.stats import power_divergence
from pgmpy.estimators.CITests import log_likelihood
from pgmpy.estimators.CITests import chi_square
# from pgmpy.factors.continuous.discretize import BaseDiscretizer

In [2]:
def csv_format_discrete(csv_file):
    df = pd.read_csv(csv_file, sep="\s+")
    return df.drop([0])

#returns the csv_file in a pandas dataframe, formatted properly, discrete dataset only

In [3]:
def csv_format_discrete(csv_file):
    df = pd.read_csv(csv_file, sep="\s+")
    return df.drop([0])


In [4]:
def gaussian_reader(csv_file):
    df = pd.read_csv(csv_file, sep="\s+")
    return df

In [5]:
#how do we measure the consistensy
def chi2bool(df):
#returns a tuple(chi2, p_value, dof) if boolean = false
#the null hypothesis is that they are independent of each other
#if true, the p_value is higher than the significance test, we do not reject the null hypothesis
#if false, the p_value is lower than the significance test, we reject the null hypothesis
    v = list(df)
    empty = []
    empty_1 = []
    empty_2 = []
    empty_3 = []
    empty_4 = []
    itertools_combinations = list(itertools.combinations(v, 2)) #finds every possible combination of list(df)
    for i in itertools_combinations:
        empty_1.append(i[0])
        empty_2.append(i[1])
        
    total_number_of_combinations = len(empty_1)
    
    for t in range(len(itertools_combinations)):
        empty.append(itertools_combinations[t])
        
    for value in range(len(empty_1)):
        empty_3.append(value)
        
    for x,y,i,j in zip(empty_1, empty_2, empty, empty_3):
        chisquare = chi_square(X=x, Y=y, Z=[], data=df, significance_level=0.05, boolean=True) #returns chi, p_value, dof
        empty_4.append([j, i , chisquare])
        
    return empty_4

In [6]:
def chi2val(df):
#returns a tuple(chi2, p_value, dof) if boolean = false
#the null hypothesis is that they are independent of each other
#if true, the p_value is higher than the significance test, we do not reject the null hypothesis
#if false, the p_value is lower than the significance test, we reject the null hypothesis
    v = list(df)
    empty = []
    empty_1 = []
    empty_2 = []
    empty_3 = []
    empty_4 = []
    itertools_combinations = list(itertools.combinations(v, 2)) #finds every possible combination of list(df)
    for i in itertools_combinations:
        empty_1.append(i[0])
        empty_2.append(i[1])
        
    total_number_of_combinations = len(empty_1)
    
    for t in range(len(itertools_combinations)):
        empty.append(itertools_combinations[t])
        
    for value in range(len(empty_1)):
        empty_3.append(value)
        
    for x,y,i,j in zip(empty_1, empty_2, empty, empty_3):
        chisquare = chi_square(X=x, Y=y, Z=[], data=df, significance_level=0.05, boolean=False) #returns chi, p_value, dof
        empty_4.append([j, i , chisquare])
        
    return empty_4


In [7]:
# def g2bool(df):
    

In [8]:
def g2(df):
#returns a tuple(chi2, p_value, dof) if boolean = false
#the null hypothesis is that they are independent of each other
#if true, the p_value is higher than the significance test, we do not reject the null hypothesis
#if false, the p_value is lower than the significance test, we reject the null hypothesis
    v = list(df)
    empty = []
    empty_1 = []
    empty_2 = []
    empty_3 = []
    empty_4 = []
    itertools_combinations = list(itertools.combinations(v, 2)) #finds every possible combination of list(df)
    for i in itertools_combinations:
        empty_1.append(i[0])
        empty_2.append(i[1])
        
    total_number_of_combinations = len(empty_1)
    
    for t in range(len(itertools_combinations)):
        empty.append(itertools_combinations[t])
        
    for value in range(len(empty_1)):
        empty_3.append(value)
        
    for x,y,i,j in zip(empty_1, empty_2, empty, empty_3):
        g2 = log_likelihood(X=x, Y=y, Z=[], significance_level=0.05, boolean=True, data=df) 
        empty_4.append([j, i , g2])
        
    return empty_4

In [9]:
def chi2return(df):
    #just a sorting function
    true_list = []
    false_list = []
    chi2 = chi2bool(df)
    for i in chi2:
        if True in i:
            true_list.append(i)
        if False in i:
            false_list.append(i)
    return false_list

In [10]:
def g2return(df):
    true_list = []
    false_list = []
    g2_return = g2(df)
    for i in g2_return:
        if True in i:
            true_list.append(i)
        if False in i:
            false_list.append(i)
    return false_list

### 

In [174]:
def chi2cond(df):
    test_list_0 = []
    test_list_1 = []
    test_list_2 = []
    chi2 = []
    v = list(df)
    combine = list(itertools.combinations(v, 3))
    for i in combine:
        test_list_0.append(i[0])
        test_list_1.append(i[1])
        test_list_2.append(i[2])
    for a,b,c in zip(test_list_0, test_list_1, test_list_2):
        chisquare = chi_square(X=a, Y=b, Z=[c], data=df, significance_level=0.05, boolean=True)
        chi2.append([a,b,c, chisquare])
    return chi2

In [ ]:
chi2cond(df)

In [154]:
df = csv_format_discrete(r"C:\Users\User\Documents\GitHub\ML_FYP\dataset\alarm_10000.dat")
df

,PULMEMBOLUS,PAP,KINKEDTUBE,INTUBATION,MINVOLSET,VENTMACH,DISCONNECT,VENTTUBE,VENTLUNG,SHUNT,...,CO,BP,EXPCO2,ERRLOWOUTPUT,PRESS,HRBP,MINVOL,HISTORY,HRSAT,PCWP
1,1,1,0,1,2,2,1,3,3,0,...,1,1,1,0,0,0,3,0,0,2
2,0,2,0,1,2,2,0,1,3,1,...,0,0,1,1,1,2,3,0,0,2
3,0,2,0,2,2,2,0,1,1,0,...,0,0,2,0,0,0,1,0,0,2
4,0,2,0,1,2,2,0,1,3,1,...,1,1,1,0,0,0,3,1,2,1
5,0,2,0,1,2,2,0,1,3,1,...,0,0,1,0,1,1,3,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,0,2,1,1,2,2,0,1,3,1,...,1,1,1,0,1,0,3,0,0,0
9997,0,0,0,1,2,2,0,1,3,1,...,0,0,1,0,2,0,3,0,0,2
9998,0,2,0,0,0,0,0,0,2,1,...,0,0,2,0,0,0,3,0,0,2
9999,0,2,0,1,2,2,0,1,3,1,...,1,1,1,0,1,0,3,0,0,0


In [162]:
test_list_0 = []
test_list_1 = []
test_list_2 = []
j = list(itertools.combinations(v, 3))
# delta = (j[0:10])
for i in j:
    test_list_0.append(i[0])
    test_list_1.append(i[1])
    test_list_2.append(i[2])
    # print(len(j)) has 20 elements

In [173]:
# test_list_0,test_list_1,test_list_2

In [165]:
for a,b,c in zip(test_list_0, test_list_1, test_list_2):
    chisquare = chi_square(X=a, Y=b, Z=[c], data=df, significance_level=0.05, boolean=True)
    print(a,b,c, chisquare)
#     print(a,b,c)

PULMEMBOLUS PAP KINKEDTUBE False
PULMEMBOLUS PAP INTUBATION False
PULMEMBOLUS PAP MINVOLSET False
PULMEMBOLUS PAP VENTMACH False
PULMEMBOLUS PAP DISCONNECT False
PULMEMBOLUS PAP VENTTUBE False
PULMEMBOLUS PAP VENTLUNG False
PULMEMBOLUS PAP SHUNT False
PULMEMBOLUS PAP VENTALV False
PULMEMBOLUS PAP FIO2 False
PULMEMBOLUS PAP PVSAT False
PULMEMBOLUS PAP SAO2 False
PULMEMBOLUS PAP INSUFFANESTH False
PULMEMBOLUS PAP ARTCO2 False
PULMEMBOLUS PAP ANAPHYLAXIS False
PULMEMBOLUS PAP TPR False
PULMEMBOLUS PAP CATECHOL False
PULMEMBOLUS PAP HR False
PULMEMBOLUS PAP ERRCAUTER False
PULMEMBOLUS PAP HREKG False
PULMEMBOLUS PAP HYPOVOLEMIA False
PULMEMBOLUS PAP LVFAILURE False
PULMEMBOLUS PAP LVEDVOLUME False
PULMEMBOLUS PAP CVP False
PULMEMBOLUS PAP STROKEVOLUME False
PULMEMBOLUS PAP CO False
PULMEMBOLUS PAP BP False
PULMEMBOLUS PAP EXPCO2 False
PULMEMBOLUS PAP ERRLOWOUTPUT False
PULMEMBOLUS PAP PRESS False
PULMEMBOLUS PAP HRBP False
PULMEMBOLUS PAP MINVOL False
PULMEMBOLUS PAP HISTORY False
PULMEMBO

PULMEMBOLUS VENTLUNG CO True
PULMEMBOLUS VENTLUNG BP True
PULMEMBOLUS VENTLUNG EXPCO2 True
PULMEMBOLUS VENTLUNG ERRLOWOUTPUT True
PULMEMBOLUS VENTLUNG PRESS True
PULMEMBOLUS VENTLUNG HRBP True
PULMEMBOLUS VENTLUNG MINVOL True
PULMEMBOLUS VENTLUNG HISTORY True
PULMEMBOLUS VENTLUNG HRSAT True
PULMEMBOLUS VENTLUNG PCWP True
PULMEMBOLUS SHUNT VENTALV False
PULMEMBOLUS SHUNT FIO2 False
PULMEMBOLUS SHUNT PVSAT False
PULMEMBOLUS SHUNT SAO2 False
PULMEMBOLUS SHUNT INSUFFANESTH False
PULMEMBOLUS SHUNT ARTCO2 False
PULMEMBOLUS SHUNT ANAPHYLAXIS False
PULMEMBOLUS SHUNT TPR False
PULMEMBOLUS SHUNT CATECHOL False
PULMEMBOLUS SHUNT HR False
PULMEMBOLUS SHUNT ERRCAUTER False
PULMEMBOLUS SHUNT HREKG False
PULMEMBOLUS SHUNT HYPOVOLEMIA False
PULMEMBOLUS SHUNT LVFAILURE False
PULMEMBOLUS SHUNT LVEDVOLUME False
PULMEMBOLUS SHUNT CVP False
PULMEMBOLUS SHUNT STROKEVOLUME False
PULMEMBOLUS SHUNT CO False
PULMEMBOLUS SHUNT BP False
PULMEMBOLUS SHUNT EXPCO2 False
PULMEMBOLUS SHUNT ERRLOWOUTPUT False
PULMEMBOL

PULMEMBOLUS ERRCAUTER MINVOL True
PULMEMBOLUS ERRCAUTER HISTORY True
PULMEMBOLUS ERRCAUTER HRSAT True
PULMEMBOLUS ERRCAUTER PCWP True
PULMEMBOLUS HREKG HYPOVOLEMIA True
PULMEMBOLUS HREKG LVFAILURE True
PULMEMBOLUS HREKG LVEDVOLUME True
PULMEMBOLUS HREKG CVP True
PULMEMBOLUS HREKG STROKEVOLUME True
PULMEMBOLUS HREKG CO True
PULMEMBOLUS HREKG BP True
PULMEMBOLUS HREKG EXPCO2 True
PULMEMBOLUS HREKG ERRLOWOUTPUT True
PULMEMBOLUS HREKG PRESS True
PULMEMBOLUS HREKG HRBP True
PULMEMBOLUS HREKG MINVOL True
PULMEMBOLUS HREKG HISTORY True
PULMEMBOLUS HREKG HRSAT True
PULMEMBOLUS HREKG PCWP True
PULMEMBOLUS HYPOVOLEMIA LVFAILURE True
PULMEMBOLUS HYPOVOLEMIA LVEDVOLUME True
PULMEMBOLUS HYPOVOLEMIA CVP True
PULMEMBOLUS HYPOVOLEMIA STROKEVOLUME True
PULMEMBOLUS HYPOVOLEMIA CO True
PULMEMBOLUS HYPOVOLEMIA BP True
PULMEMBOLUS HYPOVOLEMIA EXPCO2 True
PULMEMBOLUS HYPOVOLEMIA ERRLOWOUTPUT True
PULMEMBOLUS HYPOVOLEMIA PRESS True
PULMEMBOLUS HYPOVOLEMIA HRBP True
PULMEMBOLUS HYPOVOLEMIA MINVOL True
PULMEMB

PAP VENTTUBE INSUFFANESTH True
PAP VENTTUBE ARTCO2 True
PAP VENTTUBE ANAPHYLAXIS True
PAP VENTTUBE TPR True
PAP VENTTUBE CATECHOL True
PAP VENTTUBE HR True
PAP VENTTUBE ERRCAUTER True
PAP VENTTUBE HREKG True
PAP VENTTUBE HYPOVOLEMIA False
PAP VENTTUBE LVFAILURE True
PAP VENTTUBE LVEDVOLUME False
PAP VENTTUBE CVP False
PAP VENTTUBE STROKEVOLUME True
PAP VENTTUBE CO True
PAP VENTTUBE BP False
PAP VENTTUBE EXPCO2 True
PAP VENTTUBE ERRLOWOUTPUT True
PAP VENTTUBE PRESS True
PAP VENTTUBE HRBP True
PAP VENTTUBE MINVOL False
PAP VENTTUBE HISTORY True
PAP VENTTUBE HRSAT True
PAP VENTTUBE PCWP False
PAP VENTLUNG SHUNT True
PAP VENTLUNG VENTALV True
PAP VENTLUNG FIO2 True
PAP VENTLUNG PVSAT True
PAP VENTLUNG SAO2 True
PAP VENTLUNG INSUFFANESTH True
PAP VENTLUNG ARTCO2 True
PAP VENTLUNG ANAPHYLAXIS True
PAP VENTLUNG TPR True
PAP VENTLUNG CATECHOL True
PAP VENTLUNG HR True
PAP VENTLUNG ERRCAUTER True
PAP VENTLUNG HREKG True
PAP VENTLUNG HYPOVOLEMIA True
PAP VENTLUNG LVFAILURE True
PAP VENTLUNG LVED

PAP LVFAILURE CVP True
PAP LVFAILURE STROKEVOLUME True
PAP LVFAILURE CO True
PAP LVFAILURE BP True
PAP LVFAILURE EXPCO2 True
PAP LVFAILURE ERRLOWOUTPUT True
PAP LVFAILURE PRESS True
PAP LVFAILURE HRBP True
PAP LVFAILURE MINVOL True
PAP LVFAILURE HISTORY True
PAP LVFAILURE HRSAT True
PAP LVFAILURE PCWP True
PAP LVEDVOLUME CVP True
PAP LVEDVOLUME STROKEVOLUME True
PAP LVEDVOLUME CO True
PAP LVEDVOLUME BP True
PAP LVEDVOLUME EXPCO2 True
PAP LVEDVOLUME ERRLOWOUTPUT True
PAP LVEDVOLUME PRESS True
PAP LVEDVOLUME HRBP True
PAP LVEDVOLUME MINVOL True
PAP LVEDVOLUME HISTORY True
PAP LVEDVOLUME HRSAT True
PAP LVEDVOLUME PCWP True
PAP CVP STROKEVOLUME True
PAP CVP CO True
PAP CVP BP True
PAP CVP EXPCO2 True
PAP CVP ERRLOWOUTPUT True
PAP CVP PRESS True
PAP CVP HRBP True
PAP CVP MINVOL True
PAP CVP HISTORY True
PAP CVP HRSAT True
PAP CVP PCWP True
PAP STROKEVOLUME CO True
PAP STROKEVOLUME BP True
PAP STROKEVOLUME EXPCO2 True
PAP STROKEVOLUME ERRLOWOUTPUT True
PAP STROKEVOLUME PRESS True
PAP STROKEV

...

ARTCO2 CVP PCWP True
ARTCO2 STROKEVOLUME CO False
ARTCO2 STROKEVOLUME BP False
ARTCO2 STROKEVOLUME EXPCO2 True
ARTCO2 STROKEVOLUME ERRLOWOUTPUT False
ARTCO2 STROKEVOLUME PRESS False
ARTCO2 STROKEVOLUME HRBP True
ARTCO2 STROKEVOLUME MINVOL True
ARTCO2 STROKEVOLUME HISTORY True
ARTCO2 STROKEVOLUME HRSAT False
ARTCO2 STROKEVOLUME PCWP True
ARTCO2 CO BP False
ARTCO2 CO EXPCO2 False
ARTCO2 CO ERRLOWOUTPUT False
ARTCO2 CO PRESS False
ARTCO2 CO HRBP False
ARTCO2 CO MINVOL False
ARTCO2 CO HISTORY False
ARTCO2 CO HRSAT False
ARTCO2 CO PCWP False
ARTCO2 BP EXPCO2 False
ARTCO2 BP ERRLOWOUTPUT False
ARTCO2 BP PRESS False
ARTCO2 BP HRBP False
ARTCO2 BP MINVOL False
ARTCO2 BP HISTORY False
ARTCO2 BP HRSAT False
ARTCO2 BP PCWP False
ARTCO2 EXPCO2 ERRLOWOUTPUT False
ARTCO2 EXPCO2 PRESS False
ARTCO2 EXPCO2 HRBP False
ARTCO2 EXPCO2 MINVOL False
ARTCO2 EXPCO2 HISTORY False
ARTCO2 EXPCO2 HRSAT False
ARTCO2 EXPCO2 PCWP False
ARTCO2 ERRLOWOUTPUT PRESS True
ARTCO2 ERRLOWOUTPUT HRBP False
ARTCO2 ERRLOWOUTPUT 

TPR ERRCAUTER MINVOL True
TPR ERRCAUTER HISTORY True
TPR ERRCAUTER HRSAT False
TPR ERRCAUTER PCWP True
TPR HREKG HYPOVOLEMIA False
TPR HREKG LVFAILURE False
TPR HREKG LVEDVOLUME False
TPR HREKG CVP False
TPR HREKG STROKEVOLUME False
TPR HREKG CO False
TPR HREKG BP False
TPR HREKG EXPCO2 False
TPR HREKG ERRLOWOUTPUT False
TPR HREKG PRESS False
TPR HREKG HRBP False
TPR HREKG MINVOL False
TPR HREKG HISTORY False
TPR HREKG HRSAT False
TPR HREKG PCWP False
TPR HYPOVOLEMIA LVFAILURE True
TPR HYPOVOLEMIA LVEDVOLUME True
TPR HYPOVOLEMIA CVP True
TPR HYPOVOLEMIA STROKEVOLUME True
TPR HYPOVOLEMIA CO False
TPR HYPOVOLEMIA BP False
TPR HYPOVOLEMIA EXPCO2 True
TPR HYPOVOLEMIA ERRLOWOUTPUT True
TPR HYPOVOLEMIA PRESS True
TPR HYPOVOLEMIA HRBP True
TPR HYPOVOLEMIA MINVOL True
TPR HYPOVOLEMIA HISTORY True
TPR HYPOVOLEMIA HRSAT True
TPR HYPOVOLEMIA PCWP True
TPR LVFAILURE LVEDVOLUME True
TPR LVFAILURE CVP True
TPR LVFAILURE STROKEVOLUME True
TPR LVFAILURE CO False
TPR LVFAILURE BP False
TPR LVFAILURE EX

HR LVFAILURE EXPCO2 True
HR LVFAILURE ERRLOWOUTPUT True
HR LVFAILURE PRESS True
HR LVFAILURE HRBP True
HR LVFAILURE MINVOL True
HR LVFAILURE HISTORY True
HR LVFAILURE HRSAT True
HR LVFAILURE PCWP True
HR LVEDVOLUME CVP True
HR LVEDVOLUME STROKEVOLUME True
HR LVEDVOLUME CO False
HR LVEDVOLUME BP False
HR LVEDVOLUME EXPCO2 True
HR LVEDVOLUME ERRLOWOUTPUT True
HR LVEDVOLUME PRESS True
HR LVEDVOLUME HRBP True
HR LVEDVOLUME MINVOL True
HR LVEDVOLUME HISTORY True
HR LVEDVOLUME HRSAT True
HR LVEDVOLUME PCWP True
HR CVP STROKEVOLUME True
HR CVP CO False
HR CVP BP False
HR CVP EXPCO2 True
HR CVP ERRLOWOUTPUT True
HR CVP PRESS True
HR CVP HRBP True
HR CVP MINVOL True
HR CVP HISTORY True
HR CVP HRSAT True
HR CVP PCWP True
HR STROKEVOLUME CO False
HR STROKEVOLUME BP False
HR STROKEVOLUME EXPCO2 True
HR STROKEVOLUME ERRLOWOUTPUT True
HR STROKEVOLUME PRESS True
HR STROKEVOLUME HRBP True
HR STROKEVOLUME MINVOL True
HR STROKEVOLUME HISTORY True
HR STROKEVOLUME HRSAT True
HR STROKEVOLUME PCWP True
HR C

HYPOVOLEMIA LVFAILURE ERRLOWOUTPUT True
HYPOVOLEMIA LVFAILURE PRESS True
HYPOVOLEMIA LVFAILURE HRBP True
HYPOVOLEMIA LVFAILURE MINVOL True
HYPOVOLEMIA LVFAILURE HISTORY True
HYPOVOLEMIA LVFAILURE HRSAT True
HYPOVOLEMIA LVFAILURE PCWP False
HYPOVOLEMIA LVEDVOLUME CVP False
HYPOVOLEMIA LVEDVOLUME STROKEVOLUME False
HYPOVOLEMIA LVEDVOLUME CO False
HYPOVOLEMIA LVEDVOLUME BP False
HYPOVOLEMIA LVEDVOLUME EXPCO2 False
HYPOVOLEMIA LVEDVOLUME ERRLOWOUTPUT False
HYPOVOLEMIA LVEDVOLUME PRESS False
HYPOVOLEMIA LVEDVOLUME HRBP False
HYPOVOLEMIA LVEDVOLUME MINVOL False
HYPOVOLEMIA LVEDVOLUME HISTORY False
HYPOVOLEMIA LVEDVOLUME HRSAT False
HYPOVOLEMIA LVEDVOLUME PCWP False
HYPOVOLEMIA CVP STROKEVOLUME False
HYPOVOLEMIA CVP CO False
HYPOVOLEMIA CVP BP False
HYPOVOLEMIA CVP EXPCO2 False
HYPOVOLEMIA CVP ERRLOWOUTPUT False
HYPOVOLEMIA CVP PRESS False
HYPOVOLEMIA CVP HRBP False
HYPOVOLEMIA CVP MINVOL False
HYPOVOLEMIA CVP HISTORY False
HYPOVOLEMIA CVP HRSAT False
HYPOVOLEMIA CVP PCWP False
HYPOVOLEMIA ST

STROKEVOLUME BP PCWP False
STROKEVOLUME EXPCO2 ERRLOWOUTPUT True
STROKEVOLUME EXPCO2 PRESS True
STROKEVOLUME EXPCO2 HRBP True
STROKEVOLUME EXPCO2 MINVOL True
STROKEVOLUME EXPCO2 HISTORY True
STROKEVOLUME EXPCO2 HRSAT True
STROKEVOLUME EXPCO2 PCWP True
STROKEVOLUME ERRLOWOUTPUT PRESS True
STROKEVOLUME ERRLOWOUTPUT HRBP True
STROKEVOLUME ERRLOWOUTPUT MINVOL True
STROKEVOLUME ERRLOWOUTPUT HISTORY True
STROKEVOLUME ERRLOWOUTPUT HRSAT True
STROKEVOLUME ERRLOWOUTPUT PCWP True
STROKEVOLUME PRESS HRBP True
STROKEVOLUME PRESS MINVOL False
STROKEVOLUME PRESS HISTORY False
STROKEVOLUME PRESS HRSAT True
STROKEVOLUME PRESS PCWP True
STROKEVOLUME HRBP MINVOL True
STROKEVOLUME HRBP HISTORY True
STROKEVOLUME HRBP HRSAT True
STROKEVOLUME HRBP PCWP True
STROKEVOLUME MINVOL HISTORY True
STROKEVOLUME MINVOL HRSAT True
STROKEVOLUME MINVOL PCWP True
STROKEVOLUME HISTORY HRSAT False
STROKEVOLUME HISTORY PCWP False
STROKEVOLUME HRSAT PCWP True
CO BP EXPCO2 False
CO BP ERRLOWOUTPUT False
CO BP PRESS False
CO B

In [80]:
test_list = ['F','E','C','B']
for i,j,k in zip(range(4), range(4), range(4)):
    chisquare1 = chi_square(X=empty_1[i], Y=empty_2[j], Z=[test_list[k]], data=df, significance_level=0.05, boolean=False)
    print(chisquare1)

(2213.8813339373905, 0.0, 8)
(15.546384414951902, 0.21290658485034064, 12)
(4205.829377288974, 0.0, 12)
(7.497547119120003, 0.8230610355277854, 12)


In [66]:
for x,y,j in zip(empty_1, empty_2, test_list):
    chisquare = chi_square(X=x, Y=y, Z=[test_list], data=df, significance_level=0.05, boolean=True)
    print(chisquare)#returns chi, p_value, dof

ValueError: Grouper and axis must be same length

['A', 'B', 'C', 'D', 'E', 'F']